# Sript: API call
## File: api_call.ipynb

The script makes successive requests to the World Weather Online API (https://www.worldweatheronline.com/developer/api/historical-weather-api.aspx), based on dates for a period starting on the 1st of July of 2014 and ending on the 30th of September of 2014.

The output is a data frame (weather_data) that is saved to a csv file (weather_data.csv) in the folder Data. The retrieved information consists of:

- Date
- Maximum temperature (°C)
- Average temperature (°C)
- Minimum temperature (°C)
- UV Index
- Precipitation (mm)
- Weather description
- Cloud cover (%)
- Humidity (%)

### Functions:

In [ ]:
#### USED: https://stackoverflow.com/questions/55452835/creating-a-data-set-using-an-api-with-python

# Auxiliary function to format dates for API requests.
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)

### Creating variables

In [ ]:
# API parameters
city="New York"
start_date="2014-07-01"
end_date="2014-10-01"
date_format = "%Y-%m-%d"

# Formatting the start and end date
start_date = datetime.datetime.strptime(start_date, date_format)
end_date = datetime.datetime.strptime(end_date, date_format)

# Creating lists to store data from the API
all_data = []
date_list=[]
maxtempC_list = []
avgtempC_list = []
mintempC_list = []
uvIndex_list =[]
precipMM_list=[]
weatherDesc_list=[]
cloudcover_list=[]
humidity_list=[]

### API calls


In [ ]:
print(f"Retrieving weather information from {start_date} to {end_date}.\n")

for each_date in daterange(start_date, end_date):
#     print(each_date.date())

    ask = str(each_date.date())

    request = requests.get(
        "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?",
        params=dict(
            key=weather_api_key,
            q=city,
            format='json',
            date=ask,
            tp='24'
        )
    )
    json_data = request.json()
#     print(json_data)

    all_data.append(json_data)
    date_list.append(json_data['data']['weather'][0]['date'])
    maxtempC_list.append(json_data['data']['weather'][0]['maxtempC'])
    avgtempC_list.append(json_data['data']['weather'][0]['avgtempC'])
    mintempC_list.append(json_data['data']['weather'][0]['mintempC'])
    uvIndex_list.append(json_data['data']['weather'][0]['uvIndex'])
    precipMM_list.append(json_data['data']['weather'][0]['hourly'][0]['precipMM'])
    weatherDesc_list.append(json_data['data']['weather'][0]['hourly'][0]['weatherDesc'][0]['value'])
    cloudcover_list.append(json_data['data']['weather'][0]['hourly'][0]['cloudcover'])
    humidity_list.append(json_data['data']['weather'][0]['hourly'][0]['humidity'])

    print(f"{request}: COMPLETE ---> {json_data['data']['weather'][0]['date']}")

### Ensembling the weather data frame

In [ ]:
# create a data frame from date, maxtemp, avgtemp, uvIndex
weather_dict = {
    'date': date_list,
    "maxtempC": maxtempC_list,
    "avgtempC": avgtempC_list,
    "uvIndex": uvIndex_list,
    "precipMM":precipMM_list,
    "cloudcover":cloudcover_list,
    "humidity":humidity_list,
    "weatherDesc":weatherDesc_list    
}
weather_data = pd.DataFrame(weather_dict)
weather_data

#     weather_data.info()

# data3.Title = data3.Title.str.replace(',', '').astype(float).astype(int)
for index,column in enumerate(weather_data):
    if (index!=0) & (index!=7):
        weather_data[column]= weather_data[column].astype(float)

#     weather_data.info()

# # overwriting data after changing format
# data["Time"]= pd.to_datetime(data["Time"])
# # info of data
# data.info()
# # display
# data

weather_data["date"]=pd.to_datetime(weather_data["date"])
#     weather_data.info()

### Saving the weather data frame to a csv file

In [ ]:
weather_data.to_csv("Data/weather_data.csv", encoding = "utf-8", index = False, header = True)

print("\n\n SUCCESS!!! File 'weather_data.csv' created and saved in folder 'Data'")